In [6]:
# !pip install numpy 
# !pip install pandas 
# !pip install chromedriver_binary
# !pip install beautifulsoup4
# !pip install selenium
# !pip install webdriver-manager
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_binary
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get('https://stock-pictures.netlify.app')

In [7]:
soup  = BeautifulSoup(browser.page_source,'html.parser')

In [8]:
soup.find('img').get('src')

'https://cdn.pixabay.com/photo/2022/03/06/05/30/clouds-7050884__480.jpg'

In [9]:
data = []

for sp in soup.find_all('div',class_='container'):
    img_link = sp.find('img').get('src')
    tags = list(set(sp.find('div',class_='tags').text[7:].split(' ')))
    likes = int(sp.find(class_= 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find(class_= 'likes-comments').find_all('span')[1].text.strip()[:-8])
    print(tags)
    print(likes)
    print(comments)
    print("*"*10)
    data.append([img_link, tags, likes, comments])

df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])
df.to_csv('images.csv', index = False)


['Blue', 'Sky', 'Sky,', 'Atmosphere,', 'Clouds,']
196
55
**********
['Hummingbird', 'Bird,', 'Ornithology,']
76
20
**********
['Sea,', 'Subtropical', 'Rainfall,', 'Rainbow,']
282
106
**********
['Japan,', 'Blossoms,', 'Cherry', 'Sakura', 'Road,']
42
11
**********
['Marguerite,', 'Plant', 'Flower,', 'Cape']
39
15
**********
['Rest', 'Under', 'Tree', 'The', 'Relaxing']
522
108
**********
['Rabbit,', 'Wild', 'Cottontail', 'Grass']
51
10
**********
['Spring,', 'Plant', 'Flowers,', 'Crocus,']
135
44
**********
['Table', 'Café,', 'Vacation,', 'Drink,', 'Coffee,']
26
5
**********
['Epic,', 'Fall,', 'Light', 'Nature,', 'Waterfall,']
96
21
**********
['Nature', 'Dawn,', 'Sunrise,', 'Morning,', 'Field,']
807
131
**********
['Birdwatching,', 'Bird,', 'Robin,', 'Animal']
69
19
**********
['Bloom', 'Seedlings,', 'Flowers,', 'Soil,', 'Spring']
42
9
**********
['Hills', 'Road,', 'Highway,', 'Countryside,']
53
14
**********
['City', 'Monochrome,', 'Cityscape,', 'Bridge,']
45
13
**********
['Blossoms,'

In [10]:
df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"[Blue, Sky, Sky,, Atmosphere,, Clouds,]",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"[Hummingbird, Bird,, Ornithology,]",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"[Sea,, Subtropical, Rainfall,, Rainbow,]",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"[Japan,, Blossoms,, Cherry, Sakura, Road,]",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"[Marguerite,, Plant, Flower,, Cape]",39,15


# downloading one images 

In [11]:
link = df['img_link'][0]
# link
res = requests.get(link)
res

<Response [200]>

In [12]:
fd= open('imgg.png','wb')
fd.write(res.content)
fd.close()

# Donwloading all Images 


In [13]:

for link in df['img_link']:
    res = requests.get(link)
    fd = open('Imgs/'+ link.split('/')[-1],'wb')
    fd.write(res.content)
    fd.close()

# Infinite Scrolling 


In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
link='https://stock-pictures.netlify.app'
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get(link)


In [8]:

#Code for Infinite Scroll
for i in tqdm(range(0,540000,1000)):
	browser.execute_script("window.scrollTo(0," + str(i) + ")")  #it excepts the js scirpt to be passed as a string 
	time.sleep(.3)

 19%|███████████████▎                                                                | 103/540 [00:31<02:14,  3.25it/s]


KeyboardInterrupt: 

# Implementing dynamic Scrraping for a Website

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

# Set up the web driver and open the website
link = 'https://stock-pictures.netlify.app'
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.get(link)

# Initialize an empty list to store data
data = []

# Infinite scrolling and scraping ( start pixels, end  pixels , step)
for i in tqdm(range(0, 540000, 1000)):
    browser.execute_script(f"window.scrollTo(0, {i})")  # Scroll the page
    time.sleep(0.3)  # Short delay to allow content to load

    # Scrape the page after each scroll
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # Find all 'container' divs and extract details
    for sp in soup.find_all('div', class_='container'):
        try:
            img_link = sp.find('img').get('src')
            tags = sp.find('span', class_='tag-color').text[7:].strip()
            likes = int(sp.find('div', class_='likes-comments').find_all('span')[0].text.strip()[:-6])
            comments = int(sp.find('div', class_='likes-comments').find_all('span')[1].text.strip()[:-9])

            # Append extracted data to the list
            data.append([img_link, tags, likes, comments])

        except Exception as e:
            print(f"Error extracting data: {e}")

# Close the browser
browser.quit()

# Creating a DataFrame and saving it to a CSV file
df = pd.DataFrame(data, columns=['img_link', 'tags', 'likes', 'comments'])
df.drop_duplicates(inplace=True)  # Remove duplicate entries
df.to_csv('images.csv', index=False)

print("Scraping completed successfully. Data saved to images.csv")


 54%|█████▍    | 291/540 [02:50<02:25,  1.71it/s]


KeyboardInterrupt: 

In [14]:
len(data)

48